<div align="right" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/Logo blue_dark.png"  style="width:25px" align="right";/>
</div>

# Data anomalies –  1NF [Exercise]
© ExploreAI Academy

Database normalisation is a design technique for decoupling table structures to **reduce** data redundancies and anomalies. 

In this train, we will go through a practical example of normalising a database up to the **First Normal Form**. We will look at some of the data anomalies that can occur in practice and, at the end of the train, reflect on how **1NF** attempts to remedy their  state. 

## Learning objectives
In this notebook, we will:
* Learn the steps we can take to normalise a database up to the First Normal Form.
* Learn how to split columns and duplicate rows to enforce atomicity in a denormalised database. 
* Understand data anomalies and how database normalisation reduces the likelihood of their occurrence. 


## Imports and DB Connections

> ⚠️ ⚠️ The two exercises following this exercise (`Data anomalies – 2NF` and `Data anomalies – 3NF`) extend the concepts introduced in this exercise. Therefore, it's important to ensure that you continue using the modified `SoftDevEmployees.db` database after successfully completing each exercise.

> ⚠️ Since the queries here will modify the database, you will have to get a fresh copy to redo the code cells.


In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql

In [4]:

## Load the SoftDevEmployees database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///../db/SoftDevEmployees.db

## Data anomalies

Data anomalies are issues that present themselves in poorly structured or denormalised databases. The following are examples of commonly occurring anomalies: 

 - **Deletion anomaly**: The deletion of a record that leads to the unintentional removal of another required attribute from the database. 
 - **Insertion anomaly**: The inability to insert a record as it requires additional data that may presently not be available.
 - **Update anomaly**: This occurs when we have duplicated data; if we were to update the affected rows and a single row gets missed, this will lead to a data inconsistency.

## Denormalised database

Let us have a look at the **SoftDevEmployees.db** database which contains a **single table** called **Employees**.

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/DenoramlizedEmployeesTable.png" alt="Figure 1: Denormalised Database" >


Currently, our database is in its denormalised form. Our goal within this train is to transform this database to conform to the First Normal Form. 

## The First Normal Form – 1NF

To convert our database to the First Normal Form we need to make sure that we meet the following conditions: 

1. Each cell in the table must not hold more than one value, which is referred to as atomicity.
2. The table must have a primary key for identification.
3. It should have no duplicated rows or columns. 
4. Each column must have only one value for each row in the table.

Let us write a small query where we can see all the non-atomic items in the cells.

In [5]:
%%sql
SELECT * 
FROM 
    employees
WHERE 
    Role LIKE '%,%'    -- we use the LIKE keyword to search for the comma "," delimiter
OR 
    Department LIKE '%,%' -- we use the LIKE keyword to search for the comma "," delimiter

 * sqlite:///../db/SoftDevEmployees.db
Done.


FullName,Title,Role,OccupationBand,Salary,Department
"Dumisani, Thwala",Mr,Back-End Developer,Graduate,52171,"Web Applications, Mobile Applications"
"Dirk,Banda",Mr,Business Analyst,Intern,37601,"Web Applications,Mobile Applications"
"barend,Edwards",MR,Database Analyst,Intern,13163,"Web Applications, Mobile Applications"
"kelly ,Manuel",Ms,Full-Stack Developer,Intern,47442,"Web Applications, Mobile Applications"
"Janet,Patel",Ms,Systems Analyst,Intern,39081,"Web Applications, Mobile Applications"
"Christopher, Walker",Mr,Back-End Developer,Junior,122894,"Web Applications, Mobile Applications"
"Marco , Morris",prof,Back-End Developer,Mid-Level,110506,"Web Applications, Mobile Applications"
"Danie ,Campbell",Mrs,Business Analyst,Mid-Level,205621,"Web Applications, Mobile Applications"
"Jessica ,Mchunu",miss,"Full-Stack Developer, Scrum Master",Mid-Level,70741,Web Applications
"Laura,Makhanya",Ms,"Full-Stack Developer, Team Lead",Senior,293352,Mobile Applications


Looking at the contents of the database it is clear that the **`Employees`** table is not in the First Normal Form. The columns **`FullName`**, **`Role`**, and **`Department`** do not have single (atomic) values as required by the First Normal Form. 

## Converting to the First Normal Form
To convert the table to the First Normal Form, we will need to do two things: 

1. The first step is to reduce the content in each cell to ensure that we only store a single (atomic) value. Looking at the **`FullName`** column we see that it is in the form: **Name, Surname**  so it is logical to **split the column into two new columns.**

2. Secondly, we need to split the content for the **`Role`** and **`Department`** columns as employees can have more than one role, or belong to multiple departments. For this change, we will **duplicate the row and insert the correct 'Role' or 'Department' attribute values required.**

This will be the new structure of our database after converting to 1NF using the above steps:

<img src ="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/Practical_Normalization/1NF.png" alt="First Normal Form" >

## Exercise

### Exercise 1

Let's get to it by first creating the required table based on the above structure.

In [ ]:
# Add your code here

### Exercise 2

Before we move on to inserting values in our newly created table, let us write a few simple queries that will guide the insertion of data.

Firstly, split the contents of the `FullName` column into `Name` and `Surname`, such that each cell only contains one piece of data. Then capitalise the first letter of the `Title` column.

`Hint:` String manipulation functions will be useful here.

In [ ]:
# Add your code here

### Exercise 3

We also need to split the non-atomic values in the **`Role`** and **`Department`** columns. However, **we cannot naively split them into multiple columns** like we did for the **`Name`** column. This would require us to create `Role_1`, `Role_2`, ..., `Role_n` columns for every unique role included in each row (the same is true for the `Department` column) – which is not ideal. If we do this, we are going to potentially **introduce multiple null values** within the table. 

Furthermore, we ideally want to grow the table on a row basis instead of a column/attribute basis as this does not require a change in the table structure.

So, we will approach this problem by **creating three logical sets**. 

**3.1.** The set of all entries containing the **first Role or Department for all non-atomic cells**.

**3.2.** The set of all entries containing the **second Role or Department for all non-atomic cells**.

**3.3.** The set of **all entries** that only contain atomic cells.

**Note**: There are more efficient ways of doing this task, such as using a programming language that will inherently have more data structures available for use. However, for the purposes of this train, we will assume that SQL is the only tool available. So let us flex our SQL Ninja skills to get this done!

Let's start by getting a view of the entries that only contain **non-atomic cells**:

Continuing on the previous query in Exercise 2, edit the query to filter for all the entries (**with all of the columns included**) that only contain **non-atomic cells** in the `Role` and `Department` columns, i.e. the rows in these columns that contain commas.

In [ ]:
# Add your code here

### Exercise 3.1 – The set of all entries containing the **first** Role or Department for all non-atomic cells.

Now, let's write a query that will select the set of all entries containing the **first** Role or Department for all non-atomic cells by doing the following:

Continuing on the query in Exercise 3, edit the `Role` and `Department` columns in the `SELECT` statement to only return the values occurring before the comma, i.e. the **first** **`Role`** or **`Department`** in each non-atomic row. 

Remember to check if the row has multiple roles or departments using the `CASE` clause before applying the change.

The rest of the columns remain as they were in the original query.

In [ ]:
# Add your code here

### Exercise 3.2 – The set of all entries containing the **second** Role or Department for all non-atomic cells.

Next, let's write a query that will select the set of all entries containing the **second** Role or Department for all non-atomic cells.

In a new query continuing on the query in Exercise 3, now edit the `Role` and `Department` columns in the `SELECT` statement to only return the values occurring after the comma, i.e. the **second** **`Role`** or **`Department`** in each non-atomic row. Remember to check if the row has multiple roles or departments using the `CASE` clause before applying the change.

Again, the rest of the columns remain as they were in the original query.

In [ ]:
# Add your code here

### Exercise 3.3 – The set of **all entries** that only contain atomic cells.
Next, write a query that will select the set of **all entries** that only contain atomic cells.

In a new query, continuing on the query in Exercise 3, now edit the `Role` and `Department` columns in the `SELECT` statement to only return the values occurring after the comma, i.e. the **second** **`Role`** or **`Department`** in each non-atomic row. Remember to check if the row has multiple roles or departments using the `CASE` clause before applying the change.

Again, the rest of the columns remain as they were in the original query.

In [ ]:
# Add your code here

### Exercise 4
Write a query that will **combine together all the above sets** created in Exercise 3.1, 3.2, and 3.3 using `UNION`.

In [ ]:
# Add your code here

### Exercise 5
Use the combined query in Exercise 4 to insert the data into the table we created in Exercise 1.

In [ ]:
# Add your code here

### Checkpoint: Data anomalies

Which data anomalies do you think are still present in our database at this point of the normalisation process?


## Solutions

**`Note:` Comments have been used to explain the various lines of code in each solution.**

Take time to reflect on each of the queries given below. Feel free to play around with certain chunks of a query to cement your understanding of the various elements of each query.

### Exercise 1


In [ ]:
%%sql

DROP TABLE IF EXISTS Employees_1NF; -- We delete the table in case it exits when we create it. 

CREATE TABLE Employees_1NF (
    Name VARCHAR NOT NULL, 
    Surname VARCHAR NOT NULL,
    Role VARCHAR NOT NULL,
    Department VARCHAR NOT NULL,
    Title VARCHAR,
    OccupationBand VARCHAR,
    Salary REAL,
    PRIMARY KEY(Name, Surname, Role, Department) 
);

### Exercise 2


In [8]:
%%sql
SELECT 
    FullName,
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name, --Get substring before comma
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname, --Get substring after comma
    UPPER(SUBSTR(Title,1,1)) ||LOWER(SUBSTR(Title,2)) AS Title --Standardising all Titles to start with a capital letter
FROM 
    Employees
LIMIT 5;

 * sqlite:///../db/SoftDevEmployees.db
Done.


FullName,Name,Surname,Title
"Dumisani, Thwala",Dumisani,Thwala,Mr
"Tony, Horn",Tony,Horn,Mr
"Vuyokazi,barnes",Vuyokazi,barnes,Mr
"sello ,Details",sello,Details,Mr
"Jacqueline ,fredericks",Jacqueline,fredericks,Ms


Various string functions have been employed here. Let's unpack the `FullName` split using the first row as an example where **`FullName`** = '*Dumisani, Twala*':

```sql
    TRIM(SUBSTR('Dumisani, Twala',1,INSTR('Dumisani, Twala',,',')-1))
```
- The first function call determines the index position of the comma: **`INSTR('Dumisani, Twala',,',')`** = 9.

- The second function call returns the substring that appears before the comma: **`SUBSTR('Dumisani, Twala',1,9-1)`** = '   Dumisani'.

- The third function call removes any potential white spaces that might appear at the extremities of our substring **`TRIM('Dumisani')`** = 'Dumisani'.

The same explanation will hold true for the creation of the **`Surname`**, except that we are looking for the substring after the comma.

### Exercise 3

In [9]:
%%sql
SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,     --Splitting FullName to obtain Name,
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,    --Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) ||LOWER(SUBSTR(Title,2)) AS Title, --Standardising all Titles to start with a capital letter
    Role,
    OccupationBand,
    Salary,
    Department
FROM
    Employees
WHERE 
    ROLE LIKE '%,%' OR Department LIKE '%,%' --Targets only the non-atomic values

 * sqlite:///../db/SoftDevEmployees.db
Done.


Name,Surname,Title,Role,OccupationBand,Salary,Department
Dumisani,Thwala,Mr,Back-End Developer,Graduate,52171,"Web Applications, Mobile Applications"
Dirk,Banda,Mr,Business Analyst,Intern,37601,"Web Applications,Mobile Applications"
barend,Edwards,Mr,Database Analyst,Intern,13163,"Web Applications, Mobile Applications"
kelly,Manuel,Ms,Full-Stack Developer,Intern,47442,"Web Applications, Mobile Applications"
Janet,Patel,Ms,Systems Analyst,Intern,39081,"Web Applications, Mobile Applications"
Christopher,Walker,Mr,Back-End Developer,Junior,122894,"Web Applications, Mobile Applications"
Marco,Morris,Prof,Back-End Developer,Mid-Level,110506,"Web Applications, Mobile Applications"
Danie,Campbell,Mrs,Business Analyst,Mid-Level,205621,"Web Applications, Mobile Applications"
Jessica,Mchunu,Miss,"Full-Stack Developer, Scrum Master",Mid-Level,70741,Web Applications
Laura,Makhanya,Ms,"Full-Stack Developer, Team Lead",Senior,293352,Mobile Applications


Now that we have fixed the name columns. We can move on to the `Role` and `Department` columns. Here we get a view of all the rows in the `Role` and `Department` columns that contain **non-atomic** values. These are the rows we want to focus on.

### Exercise 3.1 – The set of all entries containing the **first** Role or Department for all non-atomic cells.

In [10]:
%%sql
SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,             -- Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,            -- Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) || LOWER(SUBSTR(Title,2)) AS Title,        -- Standardising all Titles to start with a capital letter
    
    CASE 
        WHEN                                                            -- When the row only has one role, i.e. there's no value after any comma
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))='' 
        THEN 
            Role                                                        -- return the original role
        ELSE 
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))                      -- otherwise return the substring before the comma
    END AS Role,                                                        -- and include that as the Role
    
    OccupationBand,                                                     
    Salary,                                                             
    
    CASE                                                                -- When the row only has one department, i.e. there's no value after any comma
        WHEN      
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))='' 
        THEN 
            Department                                                 -- return the original department
        ELSE 
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))         -- otherwise return the substring before the comma 
    END AS Department                                                  --  and include that as the Department
    
FROM
    Employees
WHERE 
    Role LIKE '%,%' OR Department LIKE '%,%'                           -- filter all entries that have non-atomic values in the Role and Department columns

 * sqlite:///../db/SoftDevEmployees.db
Done.


Name,Surname,Title,Role,OccupationBand,Salary,Department
Dumisani,Thwala,Mr,Back-End Developer,Graduate,52171,Web Applications
Dirk,Banda,Mr,Business Analyst,Intern,37601,Web Applications
barend,Edwards,Mr,Database Analyst,Intern,13163,Web Applications
kelly,Manuel,Ms,Full-Stack Developer,Intern,47442,Web Applications
Janet,Patel,Ms,Systems Analyst,Intern,39081,Web Applications
Christopher,Walker,Mr,Back-End Developer,Junior,122894,Web Applications
Marco,Morris,Prof,Back-End Developer,Mid-Level,110506,Web Applications
Danie,Campbell,Mrs,Business Analyst,Mid-Level,205621,Web Applications
Jessica,Mchunu,Miss,Full-Stack Developer,Mid-Level,70741,Web Applications
Laura,Makhanya,Ms,Full-Stack Developer,Senior,293352,Mobile Applications


### Exercise 3.2 – The set of all entries containing the **second** Role or Department for all non-atomic cells.

In [ ]:
%%sql
SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,             -- Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,            -- Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) || LOWER(SUBSTR(Title,2)) AS Title,        -- Standardising all Titles to start with a capital letter
    
    CASE 
        WHEN                                                            -- When the row only has one role, i.e. there's no value after any comma
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))='' 
        THEN 
            Role                                                        -- return the original role
        ELSE 
            TRIM(SUBSTR(Role,INSTR(Role,',')+1))                     -- otherwise return the substring after the comma
    END AS Role,                                                        -- and include that as the Role
    
    OccupationBand,                                                     
    Salary,                                                             
    
    CASE                                                                -- When the row only has one department, i.e. there's no value after any comma
        WHEN      
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))='' 
        THEN 
            Department                                                 -- return the original department
        ELSE 
            TRIM(SUBSTR(Department,INSTR(Department,',')+1))         -- otherwise return the substring after the comma 
    END AS Department                                                  --  and include that as the Department
    
FROM
    Employees
WHERE 
    Role LIKE '%,%' OR Department LIKE '%,%'                           -- filter all entries that have non-atomic values in the Role and Department columns

### Exercise 3.3 – The set of **all entries** that only contain atomic cells.

In [11]:
%%sql

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,     --Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,    --Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) ||LOWER(SUBSTR(Title,2)) AS Title, --Standardising all Titles to start with a capital letter
    Role,
    OccupationBand,
    Salary,
    Department
FROM
    Employees
WHERE  ROLE NOT LIKE '%,%' AND Department NOT LIKE '%,%' --Targets only the atomic values

 * sqlite:///../db/SoftDevEmployees.db
Done.


Name,Surname,Title,Role,OccupationBand,Salary,Department
Tony,Horn,Mr,Back-End Developer,Graduate,103397,Mobile Applications
Vuyokazi,barnes,Mr,Business Analyst,Graduate,69220,Web Applications
sello,Details,Mr,Database Analyst,Graduate,54945,Mobile Applications
Jacqueline,fredericks,Ms,Front-End Developer,Graduate,51104,Web Applications
Walter,Jansen,Mr,Front-End Developer,Graduate,35454,Mobile Applications
Ernest,maritz,Mr,Full-Stack Developer,Graduate,88569,Web Applications
Kevin,Motloung,Mr,Systems Analyst,Graduate,99370,Mobile Applications
takalani,Richards,Mr,Systems Analyst,Graduate,57906,Web Applications
Bronwyn,Swartz,Miss,UI/UX Developer,Graduate,34350,Mobile Applications
Jan,Ngwenya,Mr,Back-End Developer,Intern,38959,Web Applications


### Exercise 4

In [ ]:
%%sql

/*SET #1 ======================================================================================
   The set of all entries containing the first `Role` or `Department` for all non-atomic cells. 
==============================================================================================*/

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,             -- Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,            -- Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) || LOWER(SUBSTR(Title,2)) AS Title,        -- Standardising all Titles to start with a capital letter
    
    CASE 
        WHEN                                                            -- When the row only has one role, i.e. there's no value after any comma
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))='' 
        THEN 
            Role                                                        -- return the original role
        ELSE 
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))                      -- otherwise return the substring before the comma
    END AS Role,                                                        -- and include that as the Role
    
    OccupationBand,                                                     
    Salary,                                                             
    
    CASE                                                                -- When the row only has one department, i.e. there's no value after any comma
        WHEN      
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))='' 
        THEN 
            Department                                                 -- return the original department
        ELSE 
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))         -- otherwise return the substring before the comma 
    END AS Department                                                  --  and include that as the Department
    
FROM
    Employees
WHERE 
    Role LIKE '%,%' OR Department LIKE '%,%'                           -- Filter all entries that have non-atomic values in the Role and Department columns 
    
UNION

/*SET #2 ======================================================================================
   The set of all entries containing the second `Role` or `Department` for all non-atomic cells. 
==============================================================================================*/

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,             -- Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,            -- Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) || LOWER(SUBSTR(Title,2)) AS Title,        -- Standardising all Titles to start with a capital letter
    
    CASE 
        WHEN                                                            -- When the row only has one role, i.e. there's no value after any comma
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))='' 
        THEN 
            Role                                                        -- return the original role
        ELSE 
            TRIM(SUBSTR(Role,INSTR(Role,',')+1))                     -- otherwise return the substring after the comma
    END AS Role,                                                        -- and include that as the Role
    
    OccupationBand,                                                     
    Salary,                                                             
    
    CASE                                                                -- When the row only has one department, i.e. there's no value after any comma
        WHEN      
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))='' 
        THEN 
            Department                                                 -- return the original department
        ELSE 
            TRIM(SUBSTR(Department,INSTR(Department,',')+1))         -- otherwise return the substring after the comma 
    END AS Department                                                  --  and include that as the Department
    
FROM
    Employees
WHERE 
    Role LIKE '%,%' OR Department LIKE '%,%'

UNION

/*SET #3 ======================================================================================
   The set of all entries that **only** contain atomic cells. 
==============================================================================================*/

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,     --Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,    --Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) ||LOWER(SUBSTR(Title,2)) AS Title, --Standardising all Titles to start with a capital letter
    Role,
    OccupationBand,
    Salary,
    Department
FROM
    Employees
WHERE ROLE NOT LIKE '%,%' AND Department NOT LIKE '%,%' --Targets only the atomic values;

### Exercise 5

In [ ]:
%%sql
--Below is the INSERT query for the First Normal Form.

DELETE FROM Employees_1NF;

INSERT INTO Employees_1NF (Name,Surname,Title,Role,OccupationBand,Salary,Department)

/*SET #1 ======================================================================================
   The set of all entries containing the first `Role` or `Department` for all non-atomic cells. 
==============================================================================================*/

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,             -- Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,            -- Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) || LOWER(SUBSTR(Title,2)) AS Title,        -- Standardising all Titles to start with a capital letter
    
    CASE 
        WHEN                                                            -- When the row only has one role, i.e. there's no value after any comma
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))='' 
        THEN 
            Role                                                        -- return the original role
        ELSE 
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))                      -- otherwise return the substring before the comma
    END AS Role,                                                        -- and include that as the Role
    
    OccupationBand,                                                     
    Salary,                                                             
    
    CASE                                                                -- When the row only has one department, i.e. there's no value after any comma
        WHEN      
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))='' 
        THEN 
            Department                                                 -- return the original department
        ELSE 
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))         -- otherwise return the substring before the comma 
    END AS Department                                                  --  and include that as the Department
    
FROM
    Employees
WHERE 
    Role LIKE '%,%' OR Department LIKE '%,%'                           -- filter all entries that have non-atomic values in the Role and Department columns 
    
UNION

/*SET #2 ======================================================================================
   The set of all entries containing the second `Role` or `Department` for all non-atomic cells. 
==============================================================================================*/

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,             -- Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,            -- Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) || LOWER(SUBSTR(Title,2)) AS Title,        -- Standardising all Titles to start with a capital letter
    
    CASE 
        WHEN                                                            -- When the row only has one role, i.e. there's no value after any comma
            TRIM(SUBSTR(Role,1,INSTR(Role,',')-1))='' 
        THEN 
            Role                                                        -- return the original role
        ELSE 
            TRIM(SUBSTR(Role,INSTR(Role,',')+1))                     -- otherwise return the substring after the comma
    END AS Role,                                                        -- and include that as the Role
    
    OccupationBand,                                                     
    Salary,                                                             
    
    CASE                                                                -- When the row only has one department, i.e. there's no value after any comma
        WHEN      
            TRIM(SUBSTR(Department,1,INSTR(Department,',')-1))='' 
        THEN 
            Department                                                 -- return the original department
        ELSE 
            TRIM(SUBSTR(Department,INSTR(Department,',')+1))         -- otherwise return the substring after the comma 
    END AS Department                                                  --  and include that as the Department
    
FROM
    Employees
WHERE 
    Role LIKE '%,%' OR Department LIKE '%,%'

UNION

/*SET #3 ======================================================================================
   The set of all entries that **only** contain atomic cells. 
==============================================================================================*/

SELECT 
    TRIM(SUBSTR(FullName,1,INSTR(FullName,',')-1)) AS Name,     --Splitting FullName to obtain Name
    TRIM(SUBSTR(FullName,INSTR(FullName,',')+1)) AS Surname,    --Splitting FullName to obtain Surname
    UPPER(SUBSTR(Title,1,1)) ||LOWER(SUBSTR(Title,2)) AS Title, --Standardising all Titles to start with a capital letter
    Role,
    OccupationBand,
    Salary,
    Department
FROM
    Employees
WHERE ROLE NOT LIKE '%,%' AND Department NOT LIKE '%,%' --Targets only the atomic values;

## Conclusion

We can think of 1NF as the "*common sense*" form which will allow you to write meaningful SQL queries without too much hassle.

Although we have transformed the table to its First Normal Form, data anomalies still exist: 

 - **Deletion anomaly**: If we delete Jessica Mchunu from the table, the **Scrum Master** role will be removed from the database as well.

 - **Update anomaly**: Christopher's name appears twice in the table. If he were to get a raise and only one entry was updated and the other one missed, it would cause a data inconsistency – making it seem as if he is getting two different salaries.

 - **Insertion anomaly**: Some companies like to hire talent but do not necessarily assign them to a department or role as they want them to rotate throughout the company until they find their niche. This database will not allow them to capture that information as it is required that all employees belong to a department and have at least one role.



By normalising our database up to the **First Normal Form (1NF)**, we addressed some basic aspects of data organisation such as atomicity and unique identification. This provides a great foundation for further normalisation.

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>